In [1]:
%matplotlib inline

Within-subject SVM classification based on beta weights (per trials) averaged within networks from different grains of MIST parcellation, for CIMAQ memory encoding task (fMRI data).
Mean network betas reflect the engagement of a particular network for each trial.
MIST Parcellations include: 7, 12, 20, 36, 64, 122, 197, 325, 444 networks

Trials (conditions) are classifierd according to:
- task condition (encoding or control task)
- memory performance (hit vs miss, correct vs incorrect source)
- stimulus category (?)

Each model is ran and tested on data from the same subject, and then group statistics (confidence intervals) are computed around accuracy scores from each individual participant.

In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import nilearn
import scipy
import nibabel as nb
import sklearn
import seaborn as sns
import itertools

from numpy import nan as NaN
from matplotlib import pyplot as plt
from nilearn import image, plotting
from nilearn import masking
from nilearn import plotting
from nilearn import datasets
from nilearn.plotting import plot_stat_map, plot_roi, plot_anat, plot_img, show
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import MinMaxScaler


/home/fnadeau/myenv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
import ast
import gzip
import io
import json
import nilearn
import os
import sklearn
import tarfile
import tempfile
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sns
from bids_validator import BIDSValidator

from fetch_difumo import fetch_difumo
from io import BytesIO
from pathlib import Path
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join as pjoin
from pandas import DataFrame as df
from tempfile import TemporaryDirectory as tmpdir
from tempfile import TemporaryFile as tmpfile
from tqdm import tqdm
from typing import Union
from collections.abc import Iterable
from typing import Sequence

import loadutils as lu
import sniffbytes as snif
import scanzip as szip
import shutil

from nilearn import masking
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, plot_epi
from nilearn.image import concat_imgs, mean_img
from nilearn.input_data import NiftiMasker

def lst_intersection(lst1, lst2):
    '''
    Source: https://www.geeksforgeeks.org/python-intersection-two-lists/
    '''
    return [value for value in lst1 if value in set(lst2)]

def read_json(fpath:Union[str, os.PathLike]) -> dict:
    with open(fpath, mode='r', encoding='UTF-8') as jfile:
        jdict = json.load(jfile)
    jfile.close()
    return jdict
        
# with tmpfile(prefix=os.getcwd()+"/",
#             dir=os.path.splitext(bname(src_path))[0]+"/",
#             suffix=os.path.splitext(topname)[0]) as adir:
#     print(adir.name)
# tardir = xpu('~/../../data/cisl/DATA/cimaq_20190901/sourcedata')
# topname = 'cimaq_dicoms_20190901_v3.tar.gz'
# member = 'seriesUID_qc.tsv'
# tardir = '/media/francois/seagate_1tb/cimaq_dicoms_20190901'
# help(tempfile)
# src_path = pjoin(tardir, topname)
cimaq_nov_dir = xpu('~/../../data/cisl/DATA/cimaq_20190901')
cimaq_mar_dir = xpu('~/../../data/cisl/DATA/cimaq_03-19')
events_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_events/events')
behav_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_behavioural/behavioural')
participants_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'participants.json'))
dataset_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'dataset_description.json'))
# post_scan_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/task_files/PostScanBehav_CIMAQ_memory.json'))
# taskfile_headers = read_json(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/task_files/TaskFile_headers_CIMAQ_memory.json'))
# sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/features/beta_maps')))
task_results = pd.read_csv(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/TaskResults/fMRI_behavMemoScores.tsv'),
                           sep='\t')
MemoTaskParticipantFile = pd.read_csv(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/MemoTaskParticipantFile.tsv'),
                           sep='\t')
# MemoTaskParticipantFile
# task_results.columns
# sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/')))


In [8]:
sub_list_TaskQC=pd.read_csv(pjoin(cimaq_mar_dir,
                                  'derivatives/CIMAQ_fmri_memory/data/participants/sub_list_TaskQC.tsv'),
                            sep='\t')
qc_list=lu.flatten(sub_list_TaskQC.values.tolist())

In [9]:
MemoTaskParticipantFile['QC_status']=['F' if row[1].participant_id
                                      not in qc_list else 'Pass'
                                      for row in MemoTaskParticipantFile.iterrows()]
MemoTaskParticipantFile

,participant_id,cognitive_status,clinical_age_months,schooling,sex,APOE_carrier,task_randomization,QC_status,total_frames,total_scrubbed_frames,...,rt_reco_Hit,rt_reco_Miss,rt_reco_New,rt_reco_Old,rt_reco_wrongSource,rt_reco_correctSource,rt_source_FA,rt_source_Hit,rt_source_wrongSource,rt_source_correctSource
0,108391,SCD,843.6,21.0,2,0.0,B,Pass,310,131,...,1.621476,2.450333,2.582385,1.780872,1.827200,1.525791,16.804000,2.655714,4.398050,1.845326
1,120839,Controls,940.8,14.0,2,0.0,D,Pass,310,50,...,2.158792,2.985500,2.069487,2.222385,2.610000,2.086016,1.103667,0.862556,1.914000,0.692968
2,122922,SCD,894.0,18.0,1,1.0,D,Pass,310,6,...,2.359636,2.163913,2.191769,2.301923,2.589933,2.083280,4.812400,2.986782,3.402033,2.488480
3,127228,SCD,879.6,14.0,2,0.0,B,Pass,310,197,...,2.238826,2.510000,2.228667,2.270115,2.302071,2.222727,2.056000,2.226290,3.569929,1.884273
4,139593,MCI,906.0,16.0,2,0.0,C,Pass,310,75,...,2.522919,2.654415,2.499128,2.592038,2.639591,2.351800,2.833889,3.400027,3.283455,3.571000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,979001,Controls,795.6,19.0,1,0.0,C,Pass,310,9,...,1.022971,1.272000,1.204538,1.054897,1.097100,1.010190,0.895000,0.651074,0.844500,0.617724
99,983291,SCD,796.8,16.0,2,0.0,A,Pass,310,77,...,2.944533,3.128722,2.289667,2.987038,5.110400,2.511360,0.471000,0.606533,1.027800,0.522280
100,988602,SCD,813.6,17.0,2,1.0,D,Pass,310,41,...,1.219658,1.734000,1.361000,1.232846,1.348091,1.197923,0.633250,0.495447,0.679545,0.464292
101,996599,MCI,820.8,13.0,2,1.0,A,Pass,310,48,...,1.839373,3.161818,2.172077,2.025872,1.978560,1.756524,1.797375,1.494627,1.393880,1.554595


Step 1: import list of participants, and generate sublists of participants who have enough trials per category for each classification.

Encoding vs Control task conditions (all 94)
Stimulus category (all 94)
Hit versus Miss (42 participants; at least 15 trials per condition)
Correct Source versus Wrong Source (49 participants; at least 15 trials per condition)
Correct Source versus Miss (38 participants; at least 15 trials per condition)
*NOTE: ADD filter to exclude participants with too many scrubbed frames?? *

In [10]:

# Path to directory with participant lists
# data_file = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Participants/Splitting/Sub_list.tsv'
# sub_data = pd.read_csv(data_file, sep = '\t')
sub_data=MemoTaskParticipantFile
# Exclude participants who failed QC
sub_data = sub_data[sub_data['QC_status']!= 'F']

## ADD filter to exclude participants with too many scrubbed frames?? ##

# Set minimal number of trials needed per subject to include them in analysis
num = 14

# Encoding vs Control, and Stimulus Category classifications
all_subs = sub_data['participant_id']
all_diagnosis = sub_data['cognitive_status']
print(all_subs)
print(len(all_subs))

# Hit versus Miss
hm_data = sub_data[sub_data['hits'] > num]
hm_data = hm_data[hm_data['miss'] > num]
hm_subs = hm_data['participant_id']
hm_diagnosis = hm_data['cognitive_status']
print(hm_subs)
print(len(hm_subs))

# Correct Source versus Wrong Source 
cw_data = sub_data[sub_data['correct_source'] > num]
cw_data = cw_data[cw_data['wrong_source'] > num]
cw_subs = cw_data['participant_id']
cw_diagnosis = cw_data['cognitive_status']
print(cw_subs)
print(len(cw_subs))

# Correct Source versus Miss
cmiss_data = sub_data[sub_data['correct_source'] > num]
cmiss_data = cmiss_data[cmiss_data['miss'] > num]
cmiss_subs = cmiss_data['participant_id']
cmiss_diagnosis = cmiss_data['cognitive_status']
print(cmiss_subs)
print(len(cmiss_subs))


0      108391
1      120839
2      122922
3      127228
4      139593
        ...  
98     979001
99     983291
100    988602
101    996599
102    998166
Name: participant_id, Length: 94, dtype: int64
94
0      108391
2      122922
4      139593
8      164965
14     199801
17     247659
19     255499
20     258618
21     258912
24     271596
27     314409
29     336665
30     337021
36     396250
37     403131
38     408506
39     413474
41     437101
42     439776
44     458807
45     459801
47     484204
49     502616
55     567214
56     597569
60     652850
65     677561
66     711830
67     729722
68     739694
69     748676
70     763590
71     778749
72     783781
80     884343
81     886007
89     920577
91     936730
95     956130
97     974246
99     983291
102    998166
Name: participant_id, dtype: int64
42
0      108391
2      122922
4      139593
7      150649
9      175295
10     178101
12     197192
14     199801
16     229301
20     258618
22     267168
28     326073
29

In [57]:
# labels_file = '/'.join([label_dir, 'sub-'+str(sub)+'_enco_ctl.tsv'])
# labels_file
label_dir
# testlabels=pd.read_csv(labels_file,sep='\t')
# testlabels

['MIST_12.csv',
 'MIST_122.csv',
 'MIST_197.csv',
 'MIST_20.csv',
 'MIST_325.csv',
 'MIST_36.csv',
 'MIST_444.csv',
 'MIST_64.csv',
 'MIST_7.csv',
 'MIST_ROI.csv',
 'desktop.ini']

Step 2. Set up paths of directories of interest

Create empty data structures to save and export classification results


In [39]:
sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation/Release/Parcel_Information')))

['MIST_12.csv',
 'MIST_122.csv',
 'MIST_197.csv',
 'MIST_20.csv',
 'MIST_325.csv',
 'MIST_36.csv',
 'MIST_444.csv',
 'MIST_64.csv',
 'MIST_7.csv',
 'MIST_ROI.csv',
 'desktop.ini']

In [42]:
# [(item, ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data',item)))
#   for item in sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data')))]
mist_path=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation')
sorted(ls(pjoin(mist_path, 'Release/Hierarchy')))

['MIST_PARCEL_ORDER.csv', 'MIST_PARCEL_ORDER_ROI.csv', 'desktop.ini']

In [62]:
# 'sub-108391_enco_ctl.tsv'
datadir=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data')
[itm for itm in lu.loadimages(datadir)
 if bname(itm) == 'sub-108391_enco_ctl.tsv']

['/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/events/sub-108391_enco_ctl.tsv']

#### Set paths to directories of interest

In [67]:

# beta_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Betas'
# label_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Events'
# output_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nilearn/Group_results'
# basc_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcellations'
beta_dir = pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/features/beta_maps')
mist_path=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation')
# label_dir = pjoin(mist_path, 'Release/Parcel_Information')
label_dir = pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/task_files/events')
mask_dir = pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/masks')
output_dir = pjoin(dname(cimaq_mar_dir), 'cimaq_classification_outputs')
basc_dir = pjoin(cimaq_mar_dir,
                 'derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation/Release/Parcellations')

# set the parcellation level and load the parcellation map
# 7, 12, 20, 36, 64, 122, 197, 325, 444
numnets = [7, 20, 64, 325, 444]
os.mkdir(output_dir)


In [64]:
# sorted(ls())
b_labels_path=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation/Release/Parcel_Information')

Step 3. ENCODING VERSUS CONTROL TASK CLASSIFICATION

Build and test model for each participant on list, and compile data in a single pandas dataframe

In [ ]:
# ENCODING VERSUS CONTROL TASK CLASSIFICATION
numnets = [7, 20, 64, 325, 444]
for numnet in tqdm(numnets):

    basc = image.load_img(pjoin(basc_dir, 'MIST_'+str(numnet)+'.nii'))
    b_labels = pjoin(b_labels_path,'MIST_'+str(numnet)+'.csv')
#     b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+\
#                     str(numnet)+'.csv'
    basc_labels = pd.read_csv(b_labels, sep=';')

    # build data structure to store accuracy data and coefficients
    enc_ctl_data = pd.DataFrame()
    enc_ctl_data.insert(loc = 0, column = 'dccid',
                        value = np.np.nan, allow_duplicates=True)
    for i in range(0, 10):
        enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'CV'+str(i+1)+'_acc',
                            value = np.nan, allow_duplicates=True)
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TrainSet_MeanCV_acc',
                        value = np.np.nan, allow_duplicates=True)
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TestSet_acc',
                        value = np.np.nan, allow_duplicates=True)
    netnames = basc_labels['name']
    for i in tqdm(range(0, numnet)):
        enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = netnames[i]+'_coef',
                            value = np.nan, allow_duplicates=True)

    for sub in tqdm(all_subs):
        print(sub)
        s_data = [sub]
        # load subject's beta maps (one per trial)
        betas = image.load_img(img=pjoin(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                               wildcards=True)
        # initialize NiftiLabelMasker object    
        sub_mask = nb.load(pjoin(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
        sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                             memory = 'nilearn_cache', verbose=0)

        # transform subject's beta maps into vector of network means per trial
        X_enc_ctl = sub_label_masker.fit_transform(betas)

        # load subject's trial labels
        labels_file = pjoin(label_dir, 'sub-'+str(sub)+'_enco_ctl.tsv')
        enco_ctl_labels = pd.read_csv(labels_file, sep='\t')
        y_enco_ctl = enco_ctl_labels['condition']

        # mask data to exclude trials of no interest
        # does not apply here

        # Split trials into a training and a test set
        X_train, X_test, y_train, y_test = train_test_split(
            X_enc_ctl, # x
            y_enco_ctl, # y
            test_size = 0.4, # 60%/40% split
            shuffle = True, # shuffle dataset before splitting
            stratify = y_enco_ctl, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            ) 
        print('training:', len(X_train), 'testing:', len(X_test))
        print(y_train.value_counts(), y_test.value_counts())

        # define the model
        sub_svc = SVC(kernel='linear', class_weight='balanced')

        # do cross-validation to evaluate model performance
        # within 10 folds of training set
        # predict
        y_pred = cross_val_predict(sub_svc, X_train, y_train,
                                   groups=y_train, cv=10)
        # scores
        cv_acc = cross_val_score(sub_svc, X_train, y_train,
                             groups=y_train, cv=10)
        print(cv_acc)

        for i in tqdm(range(0, len(cv_acc))):
            s_data.append(cv_acc[i])

        # evaluate overall model performance on training data
        overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
        overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
        print('Accuracy:',overall_acc)
        print(overall_cr)

        s_data.append(overall_acc)

        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        acc = sub_svc.score(X_test, y_test) # get accuracy

        cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
        # print results
        print('accuracy =', acc)
        print(cr)  

        s_data.append(acc)

        # get coefficients
        coef_ = sub_svc.coef_[0]
        print(coef_.shape)
        print(coef_)

        sub_basc = basc_labels.copy()
        sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)

        coef = sub_basc['coef']
        for j in tqdm(range(0, len(coef))):
            s_data.append(coef[j])

        #sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
        #print(sub_basc.iloc[0:12, 2:4])

        enc_ctl_data = enc_ctl_data.append(pd.Series(s_data, index=enc_ctl_data.columns),
                                           ignore_index=True)

    demo_data = sub_data.copy()
    demo_data.reset_index(level=None, drop=False, inplace=True)
    col_names = ['cognitive_status','total_scrubbed_frames','mean_FD','miss',
                 'correct_source','wrong_source','dprime','associative_memScore']
    cols=tuple(zip(range(1,len(col_names)),col_names)
    for col in tqdm(cols):
        enc_ctl_data.insert(loc=col[0],column=col[1],value=demo_data[col[1]],
                            allow_duplicates=True)
    enc_ctl_data.to_csv(pjoin(output_dir, 'SVC_withinSub_enc_ctl_'+str(numnet)+'networks.tsv'),
                        sep='\t', header=True, index=False)


In [65]:
# ENCODING VERSUS CONTROL TASK CLASSIFICATION
numnets = [7, 20, 64, 325, 444]
for numnet in tqdm(numnets):

    basc = image.load_img(pjoin(basc_dir, 'MIST_'+str(numnet)+'.nii'))
    b_labels = pjoin(b_labels_path,'MIST_'+str(numnet)+'.csv')
#     b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+str(numnet)+'.csv'
    basc_labels = pd.read_csv(b_labels, sep=';')

    # build data structure to store accuracy data and coefficients
    enc_ctl_data = pd.DataFrame()
    enc_ctl_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
    for i in range(0, 10):
        enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'CV'+str(i+1)+'_acc',
                            value = NaN, allow_duplicates=True)
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TrainSet_MeanCV_acc',
                        value = 'None', allow_duplicates=True)
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TestSet_acc',
                        value = 'None', allow_duplicates=True)
    netnames = basc_labels['name']
    for i in tqdm(range(0, numnet)):
        enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = netnames[i]+'_coef',
                            value = NaN, allow_duplicates=True)

    for sub in tqdm(all_subs):
        print(sub)
        s_data = [sub]
        # load subject's beta maps (one per trial)
        betas = image.load_img(img=pjoin(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                               wildcards=True)
        # initialize NiftiLabelMasker object    
        sub_mask = nb.load(pjoin(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
        sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                             memory = 'nilearn_cache', verbose=0)

        # transform subject's beta maps into vector of network means per trial
        X_enc_ctl = sub_label_masker.fit_transform(betas)

        # load subject's trial labels
        labels_file = pjoin(label_dir, 'sub-'+str(sub)+'_enco_ctl.tsv')
        enco_ctl_labels = pd.read_csv(labels_file, sep='\t')
        y_enco_ctl = enco_ctl_labels['condition']

        # mask data to exclude trials of no interest
        # does not apply here

        # Split trials into a training and a test set
        X_train, X_test, y_train, y_test = train_test_split(
            X_enc_ctl, # x
            y_enco_ctl, # y
            test_size = 0.4, # 60%/40% split
            shuffle = True, # shuffle dataset before splitting
            stratify = y_enco_ctl, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            ) 
        print('training:', len(X_train), 'testing:', len(X_test))
        print(y_train.value_counts(), y_test.value_counts())

        # define the model
        sub_svc = SVC(kernel='linear', class_weight='balanced')

        # do cross-validation to evaluate model performance
        # within 10 folds of training set
        # predict
        y_pred = cross_val_predict(sub_svc, X_train, y_train,
                                   groups=y_train, cv=10)
        # scores
        cv_acc = cross_val_score(sub_svc, X_train, y_train,
                             groups=y_train, cv=10)
        print(cv_acc)

        for i in tqdm(range(0, len(cv_acc))):
            s_data.append(cv_acc[i])

        # evaluate overall model performance on training data
        overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
        overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
        print('Accuracy:',overall_acc)
        print(overall_cr)

        s_data.append(overall_acc)

        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        acc = sub_svc.score(X_test, y_test) # get accuracy

        cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
        # print results
        print('accuracy =', acc)
        print(cr)  

        s_data.append(acc)

        # get coefficients
        coef_ = sub_svc.coef_[0]
        print(coef_.shape)
        print(coef_)

        sub_basc = basc_labels.copy()
        sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)

        coef = sub_basc['coef']
        for j in range(0, len(coef)):
            s_data.append(coef[j])

        #sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
        #print(sub_basc.iloc[0:12, 2:4])

        enc_ctl_data = enc_ctl_data.append(pd.Series(s_data, index=enc_ctl_data.columns),
                                           ignore_index=True)


    demo_data = sub_data.copy()
    demo_data.reset_index(level=None, drop=False, inplace=True)

    enc_ctl_data.insert(loc = 1, column = 'cognitive_status',
                        value = demo_data['cognitive_status'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 2, column = 'total_scrubbed_frames',
                        value = demo_data['total_scrubbed_frames'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 3, column = 'mean_FD',
                        value = demo_data['mean_FD'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 4, column = 'hits',
                        value = demo_data['hits'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 5, column = 'miss',
                        value = demo_data['miss'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 6, column = 'correct_source',
                        value = demo_data['correct_source'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 7, column = 'wrong_source',
                        value = demo_data['wrong_source'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 8, column = 'dprime',
                        value = demo_data['dprime'], allow_duplicates=True)
    enc_ctl_data.insert(loc = 9, column = 'associative_memScore',
                        value = demo_data['associative_memScore'], allow_duplicates=True)    

    enc_ctl_data.to_csv(pjoin(output_dir, 'SVC_withinSub_enc_ctl_'+str(numnet)+'networks.tsv'),
        sep='\t', header=True, index=False)


100%|███████████████████████████████████████████| 7/7 [00:00<00:00, 1957.73it/s]

  0%|                                                    | 0/94 [00:00<?, ?it/s]

108391
training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.42857143 0.28571429 0.57142857 0.42857143 0.57142857 0.57142857
 0.42857143 0.71428571 0.71428571 0.71428571]




100%|████████████████████████████████████████| 10/10 [00:00<00:00, 94254.02it/s]

  1%|▍                                           | 1/94 [00:01<02:32,  1.64s/it]

Accuracy: 0.5428571428571428
              precision    recall  f1-score   support

         CTL       0.33      0.39      0.36        23
         Enc       0.67      0.62      0.64        47

    accuracy                           0.54        70
   macro avg       0.50      0.50      0.50        70
weighted avg       0.56      0.54      0.55        70

accuracy = 0.6808510638297872
              precision    recall  f1-score   support

         CTL       0.53      0.56      0.55        16
         Enc       0.77      0.74      0.75        31

    accuracy                           0.68        47
   macro avg       0.65      0.65      0.65        47
weighted avg       0.69      0.68      0.68        47

(7,)
[-0.00979414  0.76230294 -0.38316207  0.15300677 -0.30065299  0.53401797
 -0.02227599]
120839


/home/fnadeau/myenv/lib/python3.7/site-packages/nilearn/input_data/nifti_labels_masker.py:483: UserWarning: Persisting input arguments took 0.58s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  verbose=self.verbose)


...

/home/fnadeau/myenv/lib/python3.7/site-packages/nilearn/input_data/nifti_labels_masker.py:483: UserWarning: Persisting input arguments took 0.57s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  verbose=self.verbose)


training: 67 testing: 46
Enc    44
CTL    23
Name: condition, dtype: int64 Enc    30
CTL    16
Name: condition, dtype: int64
[0.42857143 0.57142857 1.         0.85714286 0.71428571 0.57142857
 0.57142857 0.83333333 1.         0.66666667]




100%|████████████████████████████████████████| 10/10 [00:00<00:00, 65433.76it/s]

  0%|                                                     | 0/5 [17:38<?, ?it/s]


Accuracy: 0.7164179104477612
              precision    recall  f1-score   support

         CTL       0.60      0.52      0.56        23
         Enc       0.77      0.82      0.79        44

    accuracy                           0.72        67
   macro avg       0.68      0.67      0.67        67
weighted avg       0.71      0.72      0.71        67

accuracy = 0.7391304347826086
              precision    recall  f1-score   support

         CTL       0.75      0.38      0.50        16
         Enc       0.74      0.93      0.82        30

    accuracy                           0.74        46
   macro avg       0.74      0.65      0.66        46
weighted avg       0.74      0.74      0.71        46

(7,)
[-0.10185074  0.51882647 -0.20209084  0.78330652 -0.61375276  1.01965075
  1.01842513]


FileNotFoundError: [Errno 2] No such file or directory: '/home/fnadeau/../../data/cisl/DATA/cimaq_classification_outputs/SVC_withinSub_enc_ctl_7networks.tsv'

In [61]:
sorted(ls('/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/templates/MIST_parcellation/Release/Parcel_Information'))

['MIST_12.csv',
 'MIST_122.csv',
 'MIST_197.csv',
 'MIST_20.csv',
 'MIST_325.csv',
 'MIST_36.csv',
 'MIST_444.csv',
 'MIST_64.csv',
 'MIST_7.csv',
 'MIST_ROI.csv',
 'desktop.ini']

In [44]:
# HIT VERSUS MISS TRIAL CLASSIFICATION
for numnet in numnets:

    basc = image.load_img(os.path.join(basc_dir, 'MIST_'+str(numnet)+'.nii'))
    b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+str(numnet)+'.csv'
    basc_labels = pd.read_csv(b_labels, sep=';')
    
    # build data structure to store accuracy data and coefficients
    hit_miss_data = pd.DataFrame()
    hit_miss_data.insert(loc = 0, column = 'dccid',
                         value = 'None', allow_duplicates=True)
    # hit_miss_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
    for i in range(0, 7):
        hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'CV'+str(i+1)+'_acc',
                             value = NaN, allow_duplicates=True)
    hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'TrainSet_MeanCV_acc',
                         value = 'None', allow_duplicates=True)
    hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'TestSet_acc',
                         value = 'None', allow_duplicates=True)
    netnames = basc_labels['name']
    for i in range(0, numnet):
        hit_miss_data.insert(loc = hit_miss_data.shape[1], column = netnames[i]+'_coef',
                             value = NaN, allow_duplicates=True)

    for sub in hm_subs:
        print(sub)
        s_data = [sub]
        # load subject's beta maps (one per trial)
        betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                               wildcards=True)
        # initialize NiftiLabelMasker object    
        sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
        sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                             memory = 'nilearn_cache', verbose=0)

        # transform subject's beta maps into vector of network means per trial
        X_hit_miss_ctl = sub_label_masker.fit_transform(betas)   

        # load subject's trial labels
        labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_hit.tsv')
        y_hit_miss_ctl = pd.read_csv(labels_file, sep='\t')
        y_hit_miss_ctl_labels = y_hit_miss_ctl['ctl_miss_hit']
        # mask X and y data to exclude trials of no interest
        hit_miss_mask = y_hit_miss_ctl_labels.isin(['hit', 'missed'])
        y_hit_miss = y_hit_miss_ctl_labels[hit_miss_mask]      
        X_hit_miss  = X_hit_miss_ctl[hit_miss_mask]

        # Split trials into a training and a test set
        X_train, X_test, y_train, y_test = train_test_split(
            X_hit_miss, # x
            y_hit_miss, # y
            test_size = 0.4, # 60%/40% split
            shuffle = True, # shuffle dataset before splitting
            stratify = y_hit_miss, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            ) 
        print('training:', len(X_train), 'testing:', len(X_test))
        print(y_train.value_counts(), y_test.value_counts())

        # define the model
        sub_svc = SVC(kernel='linear', class_weight='balanced')

        # do cross-validation to evaluate model performance
        # within 10 folds of training set
        # predict
        y_pred = cross_val_predict(sub_svc, X_train, y_train,
                                   groups=y_train, cv=7)
        # scores
        cv_acc = cross_val_score(sub_svc, X_train, y_train,
                             groups=y_train, cv=7)
        print(cv_acc)

        for i in range(0, len(cv_acc)):
            s_data.append(cv_acc[i])

        # evaluate overall model performance on training data
        overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
        overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
        print('Accuracy:',overall_acc)
        print(overall_cr)

        s_data.append(overall_acc)

        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        acc = sub_svc.score(X_test, y_test) # get accuracy

        cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
        # print results
        print('accuracy =', acc)
        print(cr)  

        s_data.append(acc)

        # get coefficients
        coef_ = sub_svc.coef_[0]
        print(coef_.shape)
        print(coef_)

        sub_basc = basc_labels.copy()
        sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)

        coef = sub_basc['coef']
        for j in range(0, len(coef)):
            s_data.append(coef[j])

        #sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
        #print(sub_basc.iloc[0:12, 2:4])

        hit_miss_data = hit_miss_data.append(pd.Series(s_data, index=hit_miss_data.columns), ignore_index=True)

    demo_data = hm_data.copy()
    demo_data.reset_index(level=None, drop=False, inplace=True)

    hit_miss_data.insert(loc = 1, column = 'cognitive_status', value = demo_data['cognitive_status'], allow_duplicates=True)
    hit_miss_data.insert(loc = 2, column = 'total_scrubbed_frames', value = demo_data['total_scrubbed_frames'], allow_duplicates=True)
    hit_miss_data.insert(loc = 3, column = 'mean_FD', value = demo_data['mean_FD'], allow_duplicates=True)
    hit_miss_data.insert(loc = 4, column = 'hits', value = demo_data['hits'], allow_duplicates=True)
    hit_miss_data.insert(loc = 5, column = 'miss', value = demo_data['miss'], allow_duplicates=True)
    hit_miss_data.insert(loc = 6, column = 'correct_source', value = demo_data['correct_source'], allow_duplicates=True)
    hit_miss_data.insert(loc = 7, column = 'wrong_source', value = demo_data['wrong_source'], allow_duplicates=True)
    hit_miss_data.insert(loc = 8, column = 'dprime', value = demo_data['dprime'], allow_duplicates=True)
    hit_miss_data.insert(loc = 9, column = 'associative_memScore', value = demo_data['associative_memScore'], allow_duplicates=True)    

    hit_miss_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_hit_miss_'+str(numnet)+'networks.tsv'),
        sep='\t', header=True, index=False)


NameError: name 'basc_dir' is not defined

In [45]:
# CORRECT SOURCE VERSUS WRONG SOURCE TRIAL CLASSIFICATION

for numnet in numnets:

    basc = image.load_img(os.path.join(basc_dir, 'MIST_'+str(numnet)+'.nii'))
    b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+str(numnet)+'.csv'
    basc_labels = pd.read_csv(b_labels, sep=';')

    # build data structure to store accuracy data and coefficients
    cs_ws_data = pd.DataFrame()
    cs_ws_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
    # cs_ws_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
    for i in range(0, 7):
        cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
    cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
    cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
    netnames = basc_labels['name']
    for i in range(0, numnet):
        cs_ws_data.insert(loc = cs_ws_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

    for sub in cw_subs:
        print(sub)
        s_data = [sub]
        # load subject's beta maps (one per trial)
        betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                               wildcards=True)
        # initialize NiftiLabelMasker object    
        sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
        sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                             memory = 'nilearn_cache', verbose=0)

        # transform subject's beta maps into vector of network means per trial
        X_cs_ws_miss_ctl = sub_label_masker.fit_transform(betas)   

        # load subject's trial labels
        labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_ws_cs.tsv')
        y_cs_ws_miss_ctl = pd.read_csv(labels_file, sep='\t')
        y_cs_ws_miss_ctl_labels = y_cs_ws_miss_ctl['ctl_miss_ws_cs']
        # mask X and y data to exclude trials of no interest
        cs_ws_mask = y_cs_ws_miss_ctl_labels.isin(['correctsource', 'wrongsource'])
        y_cs_ws = y_cs_ws_miss_ctl_labels[cs_ws_mask]      
        X_cs_ws  = X_cs_ws_miss_ctl[cs_ws_mask]

        # Split trials into a training and a test set
        X_train, X_test, y_train, y_test = train_test_split(
            X_cs_ws, # x
            y_cs_ws, # y
            test_size = 0.4, # 60%/40% split
            shuffle = True, # shuffle dataset before splitting
            stratify = y_cs_ws, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            ) 
        print('training:', len(X_train), 'testing:', len(X_test))
        print(y_train.value_counts(), y_test.value_counts())

        # define the model
        sub_svc = SVC(kernel='linear', class_weight='balanced')

        # do cross-validation to evaluate model performance
        # within 10 folds of training set
        # predict
        y_pred = cross_val_predict(sub_svc, X_train, y_train,
                                   groups=y_train, cv=7)
        # scores
        cv_acc = cross_val_score(sub_svc, X_train, y_train,
                             groups=y_train, cv=7)
        print(cv_acc)

        for i in range(0, len(cv_acc)):
            s_data.append(cv_acc[i])

        # evaluate overall model performance on training data
        overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
        overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
        print('Accuracy:',overall_acc)
        print(overall_cr)

        s_data.append(overall_acc)

        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        acc = sub_svc.score(X_test, y_test) # get accuracy

        cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
        # print results
        print('accuracy =', acc)
        print(cr)  

        s_data.append(acc)

        # get coefficients
        coef_ = sub_svc.coef_[0]
        print(coef_.shape)
        print(coef_)

        sub_basc = basc_labels.copy()
        sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)

        coef = sub_basc['coef']
        for j in range(0, len(coef)):
            s_data.append(coef[j])

        #sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
        #print(sub_basc.iloc[0:12, 2:4])

        cs_ws_data = cs_ws_data.append(pd.Series(s_data, index=cs_ws_data.columns), ignore_index=True)

    demo_data = cw_data.copy()
    demo_data.reset_index(level=None, drop=False, inplace=True)

    cs_ws_data.insert(loc = 1, column = 'cognitive_status', value = demo_data['cognitive_status'], allow_duplicates=True)
    cs_ws_data.insert(loc = 2, column = 'total_scrubbed_frames', value = demo_data['total_scrubbed_frames'], allow_duplicates=True)
    cs_ws_data.insert(loc = 3, column = 'mean_FD', value = demo_data['mean_FD'], allow_duplicates=True)
    cs_ws_data.insert(loc = 4, column = 'hits', value = demo_data['hits'], allow_duplicates=True)
    cs_ws_data.insert(loc = 5, column = 'miss', value = demo_data['miss'], allow_duplicates=True)
    cs_ws_data.insert(loc = 6, column = 'correct_source', value = demo_data['correct_source'], allow_duplicates=True)
    cs_ws_data.insert(loc = 7, column = 'wrong_source', value = demo_data['wrong_source'], allow_duplicates=True)
    cs_ws_data.insert(loc = 8, column = 'dprime', value = demo_data['dprime'], allow_duplicates=True)
    cs_ws_data.insert(loc = 9, column = 'associative_memScore', value = demo_data['associative_memScore'], allow_duplicates=True)    

    cs_ws_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_cs_ws_'+str(numnet)+'networks.tsv'),
        sep='\t', header=True, index=False)


NameError: name 'basc_dir' is not defined

In [11]:
# CORRECT SOURCE VERSUS MISSED TRIAL CLASSIFICATION

for numnet in numnets:
    
    basc = image.load_img(os.path.join(basc_dir, 'MIST_'+str(numnet)+'.nii'))
    b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+str(numnet)+'.csv'
    basc_labels = pd.read_csv(b_labels, sep=';')

    # build data structure to store accuracy data and coefficients
    cs_miss_data = pd.DataFrame()
    cs_miss_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
    # cs_miss_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
    for i in range(0, 7):
        cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
    cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
    cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
    netnames = basc_labels['name']
    for i in range(0, numnet):
        cs_miss_data.insert(loc = cs_miss_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

    for sub in cmiss_subs:
        print(sub)
        s_data = [sub]
        # load subject's beta maps (one per trial)
        betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                               wildcards=True)
        # initialize NiftiLabelMasker object    
        sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
        sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                             memory = 'nilearn_cache', verbose=0)

        # transform subject's beta maps into vector of network means per trial
        X_cs_ws_miss_ctl = sub_label_masker.fit_transform(betas)   

        # load subject's trial labels
        labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_ws_cs.tsv')
        y_cs_ws_miss_ctl = pd.read_csv(labels_file, sep='\t')
        y_cs_ws_miss_ctl_labels = y_cs_ws_miss_ctl['ctl_miss_ws_cs']
        # mask X and y data to exclude trials of no interest
        cs_miss_mask = y_cs_ws_miss_ctl_labels.isin(['correctsource', 'missed'])
        y_cs_miss = y_cs_ws_miss_ctl_labels[cs_miss_mask]      
        X_cs_miss  = X_cs_ws_miss_ctl[cs_miss_mask]

        # Split trials into a training and a test set
        X_train, X_test, y_train, y_test = train_test_split(
            X_cs_miss, # x
            y_cs_miss, # y
            test_size = 0.4, # 60%/40% split
            shuffle = True, # shuffle dataset before splitting
            stratify = y_cs_miss, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            ) 
        print('training:', len(X_train), 'testing:', len(X_test))
        print(y_train.value_counts(), y_test.value_counts())

        # define the model
        sub_svc = SVC(kernel='linear', class_weight='balanced')

        # do cross-validation to evaluate model performance
        # within 10 folds of training set
        # predict
        y_pred = cross_val_predict(sub_svc, X_train, y_train,
                                   groups=y_train, cv=7)
        # scores
        cv_acc = cross_val_score(sub_svc, X_train, y_train,
                             groups=y_train, cv=7)
        print(cv_acc)

        for i in range(0, len(cv_acc)):
            s_data.append(cv_acc[i])

        # evaluate overall model performance on training data
        overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
        overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
        print('Accuracy:',overall_acc)
        print(overall_cr)

        s_data.append(overall_acc)

        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        acc = sub_svc.score(X_test, y_test) # get accuracy

        cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
        # print results
        print('accuracy =', acc)
        print(cr)  

        s_data.append(acc)

        # get coefficients
        coef_ = sub_svc.coef_[0]
        print(coef_.shape)
        print(coef_)

        sub_basc = basc_labels.copy()
        sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)

        coef = sub_basc['coef']
        for j in range(0, len(coef)):
            s_data.append(coef[j])

        #sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
        #print(sub_basc.iloc[0:12, 2:4])

        cs_miss_data = cs_miss_data.append(pd.Series(s_data, index=cs_miss_data.columns), ignore_index=True)

    demo_data = cmiss_data.copy()
    demo_data.reset_index(level=None, drop=False, inplace=True)

    cs_miss_data.insert(loc = 1, column = 'cognitive_status', value = demo_data['cognitive_status'], allow_duplicates=True)
    cs_miss_data.insert(loc = 2, column = 'total_scrubbed_frames', value = demo_data['total_scrubbed_frames'], allow_duplicates=True)
    cs_miss_data.insert(loc = 3, column = 'mean_FD', value = demo_data['mean_FD'], allow_duplicates=True)
    cs_miss_data.insert(loc = 4, column = 'hits', value = demo_data['hits'], allow_duplicates=True)
    cs_miss_data.insert(loc = 5, column = 'miss', value = demo_data['miss'], allow_duplicates=True)
    cs_miss_data.insert(loc = 6, column = 'correct_source', value = demo_data['correct_source'], allow_duplicates=True)
    cs_miss_data.insert(loc = 7, column = 'wrong_source', value = demo_data['wrong_source'], allow_duplicates=True)
    cs_miss_data.insert(loc = 8, column = 'dprime', value = demo_data['dprime'], allow_duplicates=True)
    cs_miss_data.insert(loc = 9, column = 'associative_memScore', value = demo_data['associative_memScore'], allow_duplicates=True)    

    cs_miss_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_cs_miss_'+str(numnet)+'networks.tsv'),
        sep='\t', header=True, index=False)


108391
training: 34 testing: 24
correctsource    25
missed            9
Name: ctl_miss_ws_cs, dtype: int64 correctsource    18
missed            6
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.5        0.6        0.4        0.5        0.75
 0.5       ]
Accuracy: 0.5588235294117647
               precision    recall  f1-score   support

correctsource       0.81      0.52      0.63        25
       missed       0.33      0.67      0.44         9

    micro avg       0.56      0.56      0.56        34
    macro avg       0.57      0.59      0.54        34
 weighted avg       0.69      0.56      0.58        34

accuracy = 0.4166666666666667
               precision    recall  f1-score   support

correctsource       0.70      0.39      0.50        18
       missed       0.21      0.50      0.30         6

    micro avg       0.42      0.42      0.42        24
    macro avg       0.46      0.44      0.40        24
 weighted avg       0.58      0.42      0.45        24

(7,)
[-0.23810046  

training: 36 testing: 25
correctsource    22
missed           14
Name: ctl_miss_ws_cs, dtype: int64 correctsource    16
missed            9
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.6        0.6        0.4        0.6        0.6
 0.8       ]
Accuracy: 0.6111111111111112
               precision    recall  f1-score   support

correctsource       0.75      0.55      0.63        22
       missed       0.50      0.71      0.59        14

    micro avg       0.61      0.61      0.61        36
    macro avg       0.62      0.63      0.61        36
 weighted avg       0.65      0.61      0.61        36

accuracy = 0.68
               precision    recall  f1-score   support

correctsource       0.79      0.69      0.73        16
       missed       0.55      0.67      0.60         9

    micro avg       0.68      0.68      0.68        25
    macro avg       0.67      0.68      0.67        25
 weighted avg       0.70      0.68      0.69        25

(7,)
[ 0.62358384 -0.82972445 -0.68194531

training: 36 testing: 25
correctsource    22
missed           14
Name: ctl_miss_ws_cs, dtype: int64 correctsource    16
missed            9
Name: ctl_miss_ws_cs, dtype: int64
[0.33333333 0.6        0.4        0.4        0.6        0.4
 0.4       ]
Accuracy: 0.4444444444444444
               precision    recall  f1-score   support

correctsource       0.55      0.50      0.52        22
       missed       0.31      0.36      0.33        14

    micro avg       0.44      0.44      0.44        36
    macro avg       0.43      0.43      0.43        36
 weighted avg       0.46      0.44      0.45        36

accuracy = 0.52
               precision    recall  f1-score   support

correctsource       0.67      0.50      0.57        16
       missed       0.38      0.56      0.45         9

    micro avg       0.52      0.52      0.52        25
    macro avg       0.53      0.53      0.51        25
 weighted avg       0.57      0.52      0.53        25

(7,)
[ 0.53917801  0.05748084 -0.55365404

training: 34 testing: 23
missed           21
correctsource    13
Name: ctl_miss_ws_cs, dtype: int64 missed           15
correctsource     8
Name: ctl_miss_ws_cs, dtype: int64
[0.4  1.   0.8  0.6  0.6  0.6  0.25]
Accuracy: 0.6176470588235294
               precision    recall  f1-score   support

correctsource       0.50      0.69      0.58        13
       missed       0.75      0.57      0.65        21

    micro avg       0.62      0.62      0.62        34
    macro avg       0.62      0.63      0.61        34
 weighted avg       0.65      0.62      0.62        34

accuracy = 0.391304347826087
               precision    recall  f1-score   support

correctsource       0.12      0.12      0.12         8
       missed       0.53      0.53      0.53        15

    micro avg       0.39      0.39      0.39        23
    macro avg       0.33      0.33      0.33        23
 weighted avg       0.39      0.39      0.39        23

(7,)
[ 0.83891223 -0.2933732   0.52697091 -0.02304787  0.7181154

training: 32 testing: 22
correctsource    17
missed           15
Name: ctl_miss_ws_cs, dtype: int64 correctsource    12
missed           10
Name: ctl_miss_ws_cs, dtype: int64
[0.33333333 0.4        0.2        0.         0.5        0.5
 0.25      ]
Accuracy: 0.3125
               precision    recall  f1-score   support

correctsource       0.33      0.29      0.31        17
       missed       0.29      0.33      0.31        15

    micro avg       0.31      0.31      0.31        32
    macro avg       0.31      0.31      0.31        32
 weighted avg       0.31      0.31      0.31        32

accuracy = 0.45454545454545453
               precision    recall  f1-score   support

correctsource       0.50      0.42      0.45        12
       missed       0.42      0.50      0.45        10

    micro avg       0.45      0.45      0.45        22
    macro avg       0.46      0.46      0.45        22
 weighted avg       0.46      0.45      0.45        22

(7,)
[ 0.84794208  0.02318715  0.35861

...

training: 36 testing: 24
missed           26
correctsource    10
Name: ctl_miss_ws_cs, dtype: int64 missed           18
correctsource     6
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.66666667 0.66666667 0.6        0.4        0.5
 0.75      ]
Accuracy: 0.6111111111111112
               precision    recall  f1-score   support

correctsource       0.30      0.30      0.30        10
       missed       0.73      0.73      0.73        26

    micro avg       0.61      0.61      0.61        36
    macro avg       0.52      0.52      0.52        36
 weighted avg       0.61      0.61      0.61        36

accuracy = 0.5416666666666666
               precision    recall  f1-score   support

correctsource       0.27      0.50      0.35         6
       missed       0.77      0.56      0.65        18

    micro avg       0.54      0.54      0.54        24
    macro avg       0.52      0.53      0.50        24
 weighted avg       0.65      0.54      0.57        24

(444,)
[-0.00872425 -0.0134

training: 34 testing: 23
correctsource    24
missed           10
Name: ctl_miss_ws_cs, dtype: int64 correctsource    16
missed            7
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.5        0.5        0.5        0.5        0.5
 0.5       ]
Accuracy: 0.5294117647058824
               precision    recall  f1-score   support

correctsource       0.65      0.71      0.68        24
       missed       0.12      0.10      0.11        10

    micro avg       0.53      0.53      0.53        34
    macro avg       0.39      0.40      0.40        34
 weighted avg       0.50      0.53      0.51        34

accuracy = 0.6956521739130435
               precision    recall  f1-score   support

correctsource       0.74      0.88      0.80        16
       missed       0.50      0.29      0.36         7

    micro avg       0.70      0.70      0.70        23
    macro avg       0.62      0.58      0.58        23
 weighted avg       0.66      0.70      0.67        23

(444,)
[ 0.02339025  0.0225

training: 39 testing: 26
correctsource    27
missed           12
Name: ctl_miss_ws_cs, dtype: int64 correctsource    18
missed            8
Name: ctl_miss_ws_cs, dtype: int64
[0.5        0.5        0.66666667 0.83333333 0.83333333 0.6
 0.25      ]
Accuracy: 0.6153846153846154
               precision    recall  f1-score   support

correctsource       0.75      0.67      0.71        27
       missed       0.40      0.50      0.44        12

    micro avg       0.62      0.62      0.62        39
    macro avg       0.57      0.58      0.58        39
 weighted avg       0.64      0.62      0.63        39

accuracy = 0.5769230769230769
               precision    recall  f1-score   support

correctsource       0.71      0.67      0.69        18
       missed       0.33      0.38      0.35         8

    micro avg       0.58      0.58      0.58        26
    macro avg       0.52      0.52      0.52        26
 weighted avg       0.59      0.58      0.58        26

(444,)
[-0.00663641  0.0198

training: 38 testing: 26
missed           20
correctsource    18
Name: ctl_miss_ws_cs, dtype: int64 missed           13
correctsource    13
Name: ctl_miss_ws_cs, dtype: int64
[0.33333333 0.66666667 0.66666667 0.5        0.8        0.6
 1.        ]
Accuracy: 0.631578947368421
               precision    recall  f1-score   support

correctsource       0.62      0.56      0.59        18
       missed       0.64      0.70      0.67        20

    micro avg       0.63      0.63      0.63        38
    macro avg       0.63      0.63      0.63        38
 weighted avg       0.63      0.63      0.63        38

accuracy = 0.5
               precision    recall  f1-score   support

correctsource       0.50      0.23      0.32        13
       missed       0.50      0.77      0.61        13

    micro avg       0.50      0.50      0.50        26
    macro avg       0.50      0.50      0.46        26
 weighted avg       0.50      0.50      0.46        26

(444,)
[-1.29707726e-02  5.26494937e-03  3.

training: 37 testing: 26
correctsource    21
missed           16
Name: ctl_miss_ws_cs, dtype: int64 correctsource    14
missed           12
Name: ctl_miss_ws_cs, dtype: int64
[0.16666667 0.5        0.4        0.6        0.6        0.8
 0.6       ]
Accuracy: 0.5135135135135135
               precision    recall  f1-score   support

correctsource       0.56      0.67      0.61        21
       missed       0.42      0.31      0.36        16

    micro avg       0.51      0.51      0.51        37
    macro avg       0.49      0.49      0.48        37
 weighted avg       0.50      0.51      0.50        37

accuracy = 0.46153846153846156
               precision    recall  f1-score   support

correctsource       0.50      0.71      0.59        14
       missed       0.33      0.17      0.22        12

    micro avg       0.46      0.46      0.46        26
    macro avg       0.42      0.44      0.41        26
 weighted avg       0.42      0.46      0.42        26

(444,)
[ 1.70457095e-02  2